In [1]:
from azureml.core import Workspace, Dataset


subscription_id = '05ed07e3-df05-46ff-8687-8555449022bb'
resource_group = 'ML-resources'
workspace_name = 'ML-resources'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='CZ-FPZ_train_data')
dataset.to_pandas_dataframe()


,Column2,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,TrName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CZ-FPZ,0.0,-260.744016,-226.484710,-192.273677,-158.101013,-123.966777,-89.870994,-55.813647,-21.794686,...,3.217963,-3.064559,-2.457499,-1.854431,-1.265309,-0.670088,-0.088719,0.498848,1.072670,1.652806
2,CZ-FPZ,1.0,-3.681267,2.660507,9.006182,15.345819,21.669502,27.997310,34.309333,40.625661,...,-10.532356,-12.705987,-12.339189,-11.963205,-11.597941,-11.233299,-10.859188,-10.495514,-10.132176,-9.759082
3,CZ-FPZ,0.0,-71.606120,-64.131611,-56.657493,-49.203859,-41.750789,-34.318354,-26.896610,-19.475612,...,4.827849,-0.357951,0.476967,1.309357,2.149125,2.976186,3.800461,4.631865,5.450322,6.265761
4,CZ-FPZ,0.0,-133.680740,-118.116366,-102.581792,-87.057179,-71.552675,-56.078396,-40.614442,-25.180896,...,2.608151,-5.374231,-4.514962,-3.648763,-2.785720,-1.935908,-1.079401,-0.236271,0.613412,1.459579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,CZ-FPZ,0.0,-122.997418,-110.224932,-97.453754,-84.704039,-71.965929,-59.239558,-46.515052,-33.812523,...,5.142774,0.721033,1.563562,2.417558,3.262976,4.119770,4.967891,5.827292,6.677922,7.539730
659,CZ-FPZ,0.0,-176.319144,-159.898675,-143.508742,-127.139496,-110.781078,-94.453602,-78.147157,-61.851821,...,4.179946,-2.313527,-1.497134,-0.684773,0.123487,0.927583,1.727457,2.523054,3.324318,4.111204
660,CZ-FPZ,1.0,-39.324483,-31.901778,-24.479401,-17.067386,-9.675750,-2.284504,5.086353,12.456830,...,8.889631,5.796779,6.172738,6.545657,6.915507,7.282262,7.655894,8.016382,8.373714,8.727878
661,CZ-FPZ,0.0,-143.138357,-94.604604,-46.129784,2.306009,50.692729,99.030381,147.309023,195.548758,...,10.681160,1.441736,2.009988,2.580733,3.153918,3.729487,4.307385,4.887550,5.469922,6.054436


In [2]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
import pandas as pd

ws = Workspace.from_config()
experiment_name = 'test'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index= [''])
outputDf.T

,
Subscription ID,05ed07e3-df05-46ff-8687-8555449022bb
Workspace,ml-resources
Resource Group,ml-resources
Location,eastus
Experiment Name,test


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "GPU-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found exsist cluster, Use it ! ")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDART_D2_V2', max_nodes=6)

    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found exsist cluster, Use it ! 
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.train.automl import AutoMLConfig
import logging


automl_settings = {
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 1,
    "max_cores_per_iteration": 1,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             primary_metric= 'AUC_weighted',
                             compute_target=compute_target,
                             blocked_models=['TensorFlowLinearClassifier'],
                             label_column_name = '0',
                             training_data = dataset,
                             n_cross_validations=2,
                             **automl_settings
                            )


In [6]:
remote_run = experiment.submit(automl_config, show_output = False)

NameError: name 'experiment' is not defined

In [5]:
from azureml.widgets import RunDetails
remote_run.wait_for_completion(show_output=True)


NameError: name 'remote_run' is not defined

In [4]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

NameError: name 'remote_run' is not defined

In [3]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

NameError: name 'remote_run' is not defined

In [2]:
# Wait for the best model explanation run to complete
from azureml.interpret import ExplanationClient
from azureml.core.run import Run

model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

# Get the best run object
best_run, fitted_model = remote_run.get_output()

NameError: name 'remote_run' is not defined

In [12]:
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=True)
exp_data = engineered_explanations.get_feature_importance_dict()
exp_data

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


{'1': 2.9673585397336733,
 '11': 0.781907549850524,
 '2': 0.3294466805453736,
 '16': 0.13722030648752578,
 '13': 0.09904104862152553,
 '18': 0.08421913492651208,
 '30': 0.06983788158473193,
 '20': 0.05692642266060196,
 '7': 0.05460187231500641,
 '19': 0.04347004612871471,
 '25': 0.03554686948453185,
 '17': 0.03288275720215399,
 '14': 0.030759556331662954,
 '6': 0.028817308650450898,
 '8': 0.02830905862150438,
 '29': 0.02783489248575864,
 '21': 0.025339144507992815,
 '5': 0.020264911157519102,
 '10': 0.017044048824108038,
 '12': 0.015918953485571336,
 '661': 0.013675753290176899,
 '3': 0.013082010618529052,
 '72': 0.012582741703025023,
 '2540': 0.01097174745131686,
 '41': 0.008140257780679372,
 '2428': 0.007992207675703205,
 '9': 0.007111175556078415,
 '2502': 0.007053885857412504,
 '45': 0.006150837214075781,
 '57': 0.005905960148763059,
 '1062': 0.005832858267939741,
 '1762': 0.0057879186915528935,
 '1631': 0.00556366025424542,
 '4': 0.005501617058488579,
 '40': 0.005380561520165076,


In [13]:
best_run, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-core, training version:1.32.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.1, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.30.0
Package:azureml-defaults, training version:1.32.0, current version:1.30.0
Package:azureml-interpret, training version:1.32.0, current version:1.30.0
Package:azureml-mlflow, training version:1.32.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.30.0
Package:azureml-telemetry, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.32.0, current ver

In [15]:
model_name = best_run.properties['model_name']

script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')